In [1]:
# Step 1: Clean package installation with proper version resolution
try:
    # Ensure pip is up to date
    !pip install -q --upgrade pip
    # Install core numerical package
    !pip install -q numpy==1.26.0
    # Install PyTorch with CUDA 11.8 support
    !pip install -q torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    # Install Hugging Face ecosystem packages (with specific versions)
    !pip install -q transformers==4.36.2
    !pip install -q accelerate==0.25.0
    !pip install -q datasets==2.14.4
    !pip install -q peft==0.15.2
    # Install sentence-transformers with compatible version
    !pip install -q sentence-transformers==2.2.2
    # Install utility packages
    !pip install -q pdfminer.six==20221105 requests==2.32.3 beautifulsoup4==4.12.2
    !pip install -q psutil huggingface-hub==0.20.3
    # Clear outputs after installation
    from IPython.display import clear_output
    clear_output()
    # Verify critical installations
    import torch
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    # Explicitly verify accelerate installation
    import accelerate
    print(f"Accelerate version: {accelerate.__version__}")
except Exception as e:
    print(f"Package installation failed: {str(e)}")
    raise



PyTorch version: 2.1.0+cu118
CUDA available: True
Accelerate version: 0.25.0


In [2]:
!pip install -q accelerate>=0.21.0 # Upgrade accelerate to at least 0.21.0
# Or, downgrade peft with
!pip install -q peft==0.5.0 # downgrade to peft==0.5.0

In [1]:
# Step 2: Import with memory monitoring and compatibility checks
import psutil
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import os
from pdfminer.high_level import extract_text
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

def print_memory():
    """Print current memory usage"""
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU: {gpu_mem:.2f}GB", end=" | ")
    ram = psutil.virtual_memory()
    print(f"RAM: {ram.percent}% ({ram.used/1024**3:.1f}/{ram.total/1024**3:.1f}GB)")

print("Initial memory:")
print_memory()

Initial memory:
GPU: 0.00GB | RAM: 16.1% (1.7/12.7GB)


In [2]:


# Step 3: Optimized arXiv fetcher with better error handling
def get_arxiv_paper(paper_id, max_pages=1):
    """Fetch paper text with memory limits"""
    try:
        # Fetch metadata
        url = f"https://arxiv.org/abs/{paper_id}"
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.find('h1', class_='title').get_text(strip=True).replace('Title:', '')
        authors = soup.find('div', class_='authors').get_text(strip=True).replace('Authors:', '')
        abstract = soup.find('blockquote', class_='abstract').get_text(strip=True).replace('Abstract:', '')

        # Fetch PDF content if memory allows
        pdf_text = ""
        if psutil.virtual_memory().percent < 90:
            try:
                pdf_url = f"https://arxiv.org/pdf/{paper_id}.pdf"
                with requests.get(pdf_url, stream=True, timeout=15) as pdf_response:
                    pdf_response.raise_for_status()
                    temp_file = f"temp_{paper_id}.pdf"
                    with open(temp_file, 'wb') as f:
                        for chunk in pdf_response.iter_content(chunk_size=8192):
                            f.write(chunk)
                    pdf_text = extract_text(temp_file, maxpages=max_pages)[:5000]
                    os.remove(temp_file)
            except Exception as e:
                print(f"⚠️ PDF processing failed for {paper_id}: {str(e)}")

        return {
            'title': title,
            'authors': authors,
            'summary': abstract,
            'content': pdf_text
        }
    except Exception as e:
        print(f"⚠️ Error fetching {paper_id}: {str(e)}")
        return None

In [3]:
# Step 4: Model loading with enhanced error handling
MODEL_NAME = "microsoft/phi-1"

# Check GPU availability
if not torch.cuda.is_available():
    print("⚠️ CUDA is not available. Using CPU (this will be very slow)")
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

# Load tokenizer and model with fallbacks
try:
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        padding_side="right"
    )
    tokenizer.pad_token = tokenizer.eos_token

    print("Loading model...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        trust_remote_code=True,
        # Remove or comment out device_map="auto"
        # device_map="auto" if device.type == "cuda" else None,
        low_cpu_mem_usage=True
    ).to(device)

    print("Model loaded successfully:")
    print_memory()
except Exception as e:
    print(f"❌ Model loading failed: {str(e)}")
    print("\nTroubleshooting steps:")
    print("1. Try: Runtime → Restart runtime → Run again")
    print("2. Use smaller model: MODEL_NAME = 'microsoft/phi-1'")
    if "out of memory" in str(e).lower():
        print("3. Reduce batch size or model size")
    raise

Loading tokenizer...
Loading model...


Some weights of the model checkpoint at microsoft/phi-1 were not used when initializing PhiForCausalLM: ['model.layers.15.self_attn.k_proj.bias', 'model.layers.21.self_attn.k_proj.weight', 'model.layers.23.self_attn.q_proj.weight', 'model.layers.23.self_attn.q_proj.bias', 'model.layers.16.self_attn.q_proj.bias', 'model.layers.22.self_attn.v_proj.weight', 'model.layers.18.self_attn.q_proj.weight', 'model.layers.12.self_attn.k_proj.weight', 'model.layers.4.self_attn.k_proj.bias', 'model.layers.3.self_attn.q_proj.weight', 'model.layers.5.self_attn.v_proj.bias', 'model.layers.7.self_attn.q_proj.bias', 'model.layers.13.self_attn.q_proj.weight', 'model.layers.10.self_attn.v_proj.weight', 'model.layers.15.self_attn.v_proj.weight', 'model.layers.8.self_attn.q_proj.weight', 'model.layers.2.self_attn.q_proj.weight', 'model.layers.19.self_attn.v_proj.weight', 'model.layers.14.self_attn.q_proj.weight', 'model.layers.12.self_attn.v_proj.bias', 'model.layers.11.self_attn.v_proj.bias', 'model.layers.

Model loaded successfully:
GPU: 2.65GB | RAM: 22.2% (2.5/12.7GB)


In [4]:
# Step 5: Revised data loading and model preparation
def tokenize_function(examples):
    # Tokenize without return_tensors="pt" first
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )

    # Create labels by copying input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

# Load a small dataset from local text
try:
    # Create a simple dataset
    sample_texts = [
        "This is a sample text for training the language model.",
        "Language models learn to predict the next word in a sequence.",
        "The quick brown fox jumps over the lazy dog."
    ]
    dataset = Dataset.from_dict({'text': sample_texts})

    # Apply tokenization (batched processing)
    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # Convert to PyTorch tensors and move to device
    tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

except Exception as e:
    print(f"❌ Dataset creation failed: {str(e)}")
    raise

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [5]:
# Step 6: Revised training setup

# First define your training arguments (add this before the safe_colab_train function)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,  # Reduced batch size
    gradient_accumulation_steps=2,
    save_steps=1000,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True if torch.cuda.is_available() else False,
    warmup_steps=100,
    save_total_limit=2,
    report_to="none"
)

# Define your PEFT config (LoRA configuration)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

def safe_colab_train(model, peft_config, tokenized_dataset, training_args):
    try:
        print_memory()
        if psutil.virtual_memory().percent > 90:
            raise RuntimeError("Insufficient RAM before training")

        # Prepare model for training
        model = prepare_model_for_kbit_training(model)
        model = get_peft_model(model, peft_config)

        # Verify gradients are enabled
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(f"Trainable parameter: {name}")

        model.print_trainable_parameters()

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset,
        )

        print("🚀 Starting training...")
        trainer.train()
        return trainer

    except Exception as e:
        print(f"❌ Training failed: {str(e)}")
        if "out of memory" in str(e).lower():
            print("Try reducing batch size or model size")
        raise

# Move model to device and start training
model = model.to(device)
print("Starting training...")
trainer = safe_colab_train(model, peft_config, tokenized_dataset, training_args)

Starting training...
GPU: 2.65GB | RAM: 22.2% (2.5/12.7GB)
Trainable parameter: base_model.model.model.layers.0.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.0.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.1.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.1.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.2.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.2.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.3.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.3.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.4.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.4.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model

Step,Training Loss


In [6]:
# Step 7: Model Evaluation and Saving
def evaluate_model(trainer, model, tokenizer):
    """Evaluate the model and print sample outputs"""
    print("\nEvaluating model...")

    # Generate sample output
    print("\nSample generations:")
    input_text = "Language models are"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=50,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Input: {input_text}")
    print(f"Generated: {generated_text}")

    # Calculate and print training metrics if available
    if hasattr(trainer, 'metrics'):
        print("\nTraining metrics:")
        for key, value in trainer.metrics.items():
            print(f"{key}: {value:.4f}")

def save_model(model, tokenizer, output_dir="./phi-lora-trained"):
    """Save the trained model and tokenizer"""
    print(f"\nSaving model to {output_dir}")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print("Model saved successfully.")


In [7]:

# Step 8: Load Saved Model (for future use)
def load_saved_model(model_path, device):
    """Load a previously saved model"""
    print(f"\nLoading model from {model_path}")

    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        trust_remote_code=True,
        padding_side="right"
    )
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        trust_remote_code=True,
        device_map="auto" if device.type == "cuda" else None
    ).to(device)

    print("Model loaded successfully.")
    return model, tokenizer


In [8]:

# Step 9: Inference Function
def generate_text(model, tokenizer, prompt, max_length=100, temperature=0.7):
    """Generate text from a prompt"""
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [9]:
# Step 10: Main Execution Flow
def main():
    try:
        # Run training
        # Pass the necessary arguments to safe_colab_train
        trainer = safe_colab_train(model, peft_config, tokenized_dataset, training_args)  # Pass the required arguments
        # Evaluate and save model
        evaluate_model(trainer, model, tokenizer)
        save_model(model, tokenizer)

        # Demonstrate inference
        print("\nRunning inference examples:")
        prompts = [
            "Artificial intelligence is",
            "The future of machine learning",
            "Researchers discovered that"
        ]

        for prompt in prompts:
            generated = generate_text(model, tokenizer, prompt)
            print(f"\nPrompt: {prompt}")
            print(f"Generated: {generated}")

        print("\nTraining and evaluation completed successfully!")

    except Exception as e:
        print(f"\nError in main execution: {str(e)}")
        raise

# Execute the main workflow
if __name__ == "__main__":
    main()

GPU: 5.31GB | RAM: 23.0% (2.5/12.7GB)
Trainable parameter: base_model.model.model.layers.0.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.0.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.1.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.1.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.2.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.2.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.3.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.3.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.4.self_attn.dense.lora_A.default.weight
Trainable parameter: base_model.model.model.layers.4.self_attn.dense.lora_B.default.weight
Trainable parameter: base_model.model.model.layers.5

Step,Training Loss



Evaluating model...

Sample generations:
Input: Language models are
Generated: Language models ares[i_tof, ranges: 0:



















Saving model to ./phi-lora-trained
Model saved successfully.

Running inference examples:

Prompt: Artificial intelligence is
Generated: Artificial intelligence isosry through alligator_: A ==========, butler'st_range_right_t_<-out_bject. Unordered to_sequences. If the function that define a = 42, buttons of theirs = distance between 1 <= limit: 1234_<theories ==========10x in descending_: 0: returns a random_douette: a =~






Prompt: The future of machine learning
Generated: The future of machine learning_


        
 of theories ============== 4
 of the middle element in: Aries ==============: A: A inp_range ============== for alligators_:= 7:=:=:=: a tuple_: a_: 3rd in between 1xericks_:







        # Define[i: 2Dictically sorted_

Prompt: Researchers discovered that
Generated: Researchers discovered that calculates the function toks aresorte